# Sistema de Recomendación de películas

- Lo que trataremos de hacer en el siguiente trabajo es recomendarle a los usuarios que han valorado películas, nuevas películas según sus gustos y valoraciones pasadas, prediciendo a través del modelo el puntaje que le otorgará el usuario a cada película no vista. Tomaremos en consideración para el análisis las películas enlistadas en el dataset 'movies' y las valoraciones de cada usuario sobre las películas en 'ratings' (de 1 a 5 estrellas).
- El sistema de recomendación será realizado con Auto Encoders, que es una herramienta que nos permite, de alguna manera, "comprimir" los datos de entrada en un tamaño menor para luego poder ser reconvertidos a sus valores iniciales nuevamente. En este proceso de resúmen y reconstrucción el algoritmo aprende lo suficiente sobre los datos como para poder predecir valores no utilizados en entrenamiento.

# Importamos bibliotecas y Datos

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
# Importamos sets
movies = pd.read_csv("ml-25m/movies.csv", sep = ',', encoding = 'latin-1') # ids de todas las películas
ratings  = pd.read_csv("ml-25m/ratings.csv", sep = ',', encoding = 'latin-1').iloc[:,:-1] # valoraciones de usuarios sobre películas vistas

In [3]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [4]:
ratings

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5
...,...,...,...
25000090,162541,50872,4.5
25000091,162541,55768,2.5
25000092,162541,56176,2.0
25000093,162541,58559,4.0


# Aplicamos Stacked Auto-Encoders 

- Inicialmente el dataset posee 25000000 valoraciones de películas de usuarios, pero por motivos de memoria y almacenamiento trabajaremos sólo con los usuarios que hayan efectuado el mayor número de opiniones y con las películas más valoradas.

#### Filtramos las valoraciones de los 10000 usuarios y 1000 películas más importantes del dataset

In [5]:
# Limitar a una muestra de usuarios y películas populares
n_users_sample = 10000  
n_movies_sample = 1000  

# Seleccionar los usuarios y películas con más valoraciones
top_users = ratings['userId'].value_counts().nlargest(n_users_sample).index
top_movies = ratings['movieId'].value_counts().nlargest(n_movies_sample).index

ratings = ratings[ratings['userId'].isin(top_users) & ratings['movieId'].isin(top_movies)].copy()

# Reasignar índices consecutivos a 'userId' y 'movieId'
ratings['userId'], unique_users = pd.factorize(ratings['userId'])
ratings['movieId'], unique_movies = pd.factorize(ratings['movieId'])

# Vemos la cantidad de valoraciones con las que nos quedamos una vez finiquitado el filtrado
print(ratings.shape)

(4027019, 3)


#### Guardamos el número de usuarios y películas

In [6]:
nb_users = len(unique_users)
nb_movies = len(unique_movies)

#### Dividimos las valoraciones filtradas en conjunto de train y de test

- Dividimos los datos en una proporción del 80/20 para el conjunto de train y test. Con esto buscaremos que el modelo se ajuste al 80% de los datos para que trate de predecir el 20% restante.

In [7]:
from sklearn.model_selection import train_test_split
training_set, test_set = train_test_split(ratings, test_size = 0.2, random_state = 0)

#### Convertir los datos en un array X[u,i] con usuarios u en fila y películas i en columna

In [8]:
# Convertir training_set y test_set a arrays de NumPy
training_set_np = np.array(training_set)
test_set_np = np.array(test_set)

# Definir la función de conversión
def convert(data, nb_users, nb_movies):
    new_data = []
    for id_user in range(nb_users):
        id_movies = data[:, 1][data[:, 0] == id_user].astype(int) 
        id_ratings = data[:, 2][data[:, 0] == id_user]
        ratings = np.zeros(nb_movies)
        ratings[id_movies] = id_ratings
        new_data.append(list(ratings))
    return new_data

# Convertir training_set_np y test_set_np a formato de matriz
training_set = convert(training_set_np, nb_users, nb_movies)
test_set = convert(test_set_np, nb_users, nb_movies)

#### Convertir los datos a tensores de Pytorch

In [9]:
# Convertir a tensores
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

#### Definimos la arquitectura del SAE (Stacked Auto Encoders)

- Para el SAE, usaremos 4 capas ocultas en donde la información pase a ser resumida a 200 neuronas, y luego a 100, para luego volver a comenzar el proceso de reconstrucción gradual de los datos de entrada. Tambien usaremos una función de activación Sigmoidal.

In [10]:
# Probar modelos
class SAE(nn.Module):
    def __init__(self, nb_movies):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 200)
        self.fc2 = nn.Linear(200, 100)
        self.fc3 = nn.Linear(100, 200)
        self.fc4 = nn.Linear(200, nb_movies)
        self.activation = nn.Sigmoid() 
        #self.dropout = nn.Dropout(0.3) # por si necesitamos aplicar Dropout

    def forward(self, x):
        x = self.activation(self.fc1(x))
        #x = self.dropout(x) # aplicar dropout entre capas
        x = self.activation(self.fc2(x))
        #x = self.dropout(x) # aplicar dropout entre capas
        x = self.activation(self.fc3(x))
        #x = self.dropout(x) # aplicar dropout entre capas
        x = self.fc4(x)  # La salida final tiene el mismo tamaño que la entrada
        return x

# Clave: cuidarse de no agregar muchas neuronas por capa para que luego no queden muertas y tiendan todas las de una misma capa al mismo valor

#### Entrenar el SAE procesando individualmente los usuarios, vemos resultados en train y test y guardamos desempeño y pérdida por época

- Usamos como función de pérdida el MSE (Error Cuadrado Medio), que nos indicará, en promedio, en cuantas estrellas de clasificación (1-5) se equivoca el modelo respecto a los datos de entrenamiento y test. Por ejemplo, si el MSE es de 0.90, el modelo se equivocará prediciendo en promedio alrededor de 0.90 estrellas sobre 5.
- Utilizamos como optimizador RMSprop, con un ratio de aprendizaje del 0.001 y un decaimiento del 0.4.
- También hemos determinado como 100 el número de épocas necesario para la convergencia.
- Cabe aclarar que estos parámetros son determinados mediante pruebas, y los que empleo actualmente son el producto de los mejores resultados que he podido obtener del modelo.

In [11]:
# Crear la instancia del modelo
sae = SAE(nb_movies)
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.001, weight_decay=0.4)

# Inicializamos de manera robusta los pesos (Xavier)
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

# Luego aplicas la función a tu modelo
sae.apply(init_weights)

nb_epoch = 100
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae.forward(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data * mean_corrector)
            s += 1.
            optimizer.step()
    print(f"Epoch: {epoch}, Loss: {train_loss / s}")

# Guardar el modelo después de cada época
    torch.save(sae.state_dict(), f'sae_epoch_{epoch}.pth')

Epoch: 1, Loss: 0.9188399314880371
Epoch: 2, Loss: 0.8846944570541382
Epoch: 3, Loss: 0.8793891668319702
Epoch: 4, Loss: 0.8763469457626343
Epoch: 5, Loss: 0.8736611604690552
Epoch: 6, Loss: 0.8721025586128235
Epoch: 7, Loss: 0.8703178763389587
Epoch: 8, Loss: 0.868691623210907
Epoch: 9, Loss: 0.8675103783607483
Epoch: 10, Loss: 0.8658542037010193
Epoch: 11, Loss: 0.8652476668357849
Epoch: 12, Loss: 0.8643067479133606
Epoch: 13, Loss: 0.8642560839653015
Epoch: 14, Loss: 0.8621439337730408
Epoch: 15, Loss: 0.8621289134025574
Epoch: 16, Loss: 0.8619673848152161
Epoch: 17, Loss: 0.8606168031692505
Epoch: 18, Loss: 0.8594791293144226
Epoch: 19, Loss: 0.8588276505470276
Epoch: 20, Loss: 0.8584209084510803
Epoch: 21, Loss: 0.8594984412193298
Epoch: 22, Loss: 0.858390212059021
Epoch: 23, Loss: 0.8577073216438293
Epoch: 24, Loss: 0.8584260940551758
Epoch: 25, Loss: 0.8579435348510742
Epoch: 26, Loss: 0.8587026596069336
Epoch: 27, Loss: 0.8584531545639038
Epoch: 28, Loss: 0.8569313287734985
Epo

#### Evaluar el SAE entrenado con el conjunto de prueba

- Ahora utilizaremos los parámetros de la época de entrenamiento que mejores resultados dieron para intentar predecir sobre el conjunto de prueba. Intentaremos que presente resultados similares al del conjunto de entrenamiento para no tener problemas de Overffiting o Underffiting.

In [13]:
# Cargar los parámetros guardados en la época 83 (mejor época)
sae.load_state_dict(torch.load('sae_epoch_83.pth', weights_only=True))
sae.eval()  # Cambiar el modelo a modo de evaluación (opcional pero recomendado)

# Medir el rendimiento en el conjunto de prueba
test_loss = 0
s = 0.
with torch.no_grad():  # Desactiva el cálculo de gradientes para la evaluación
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = Variable(test_set[id_user]).unsqueeze(0)
        if torch.sum(target.data > 0) > 0:
            output = sae.forward(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
            test_loss += np.sqrt(loss.data * mean_corrector)
            s += 1.

print(f"Test Loss: {test_loss / s}")

Test Loss: 0.849509060382843


#### Generar predicciones de valoraciones para un usuario en particular a partir del SAE entrenado

- Generamos para un usuario a elección predicciones de las posibles valoraciones que emitirá sobre las películas que no vió en base a las opiniones que tomamos de sus películas vistas.

In [16]:
# Usuario específico 
id_user = 0  

# Preparar entrada y salida para el usuario específico
input = Variable(training_set[id_user]).unsqueeze(0)
output = sae.forward(input).squeeze(0)  # Eliminar la dimensión adicional

# Máscara para identificar las películas no vistas
unseen_mask = (test_set[id_user] == 0)  # Películas no vistas por este usuario
unseen_predictions = output[unseen_mask].data.numpy()  # Predicciones para las películas no vistas
unseen_movies = np.where(unseen_mask.numpy())[0]  # Índices de las películas no vistas

# Mostrar resultados para el usuario 1
print(f"Usuario {id_user + 1}: Películas no vistas - {unseen_movies}")
print("--------------------------------------------------------------------------------------")
print(f"Predicciones - {unseen_predictions}")

Usuario 1: Películas no vistas - [  0   2   3   5   8  10  12  13  14  15  16  17  18  19  20  21  22  23
  24  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42
  44  45  46  47  48  49  50  51  52  53  54  55  56  57  59  61  63  65
  66  67  68  69  70  71  73  74  75  76  77  78  82  83  84  85  87  88
  89  90  91  95  97  99 100 101 102 104 106 108 109 110 111 113 114 116
 117 118 119 120 121 122 123 124 129 130 131 132 134 135 136 138 139 140
 145 146 148 149 150 151 152 153 154 155 156 157 158 160 161 162 163 164
 165 166 167 169 170 174 175 178 180 181 183 184 185 186 188 189 191 193
 194 195 196 197 198 199 200 201 202 203 204 205 206 207 209 212 214 215
 216 217 218 220 221 222 223 224 226 227 228 229 231 232 233 234 235 238
 239 240 241 242 244 245 246 247 248 249 251 252 253 254 255 256 257 258
 259 260 261 262 263 264 265 268 269 270 271 272 273 274 275 276 277 278
 279 281 282 285 287 288 290 291 292 293 296 298 299 300 302 303 305 307
 308 309 310 312 3

#### Generar predicciones de valoraciones para cada usuario a partir del SAE entrenado

In [ ]:
# for id_user in range(nb_users):
#    input = Variable(training_set[id_user]).unsqueeze(0)
#    output = sae.forward(input).squeeze(0)  # Eliminar la dimensión adicional
#    unseen_mask = (test_set[id_user] == 0)  # Máscara para películas no vistas
#    unseen_predictions = output[unseen_mask].data.numpy()  # Usar la máscara corregida
#    unseen_movies = np.where(unseen_mask.numpy())[0]  # Obtener los índices de películas no vistas
#    print(f"Usuario {id_user + 1}: Películas no vistas - {unseen_movies}, Predicciones - {unseen_predictions}")

#### Efectuamos un top 10 de recomendaciones para un usuario en particular

In [18]:
# Mapear índices de películas (del modelo) a movieId
movie_idx_to_id = {i: movieId for i, movieId in enumerate(movies['movieId'])}
movie_id_to_title = {movieId: title for movieId, title in zip(movies['movieId'], movies['title'])}

# Usuario específico (por ejemplo, el usuario con id 259)
user_id = 259

# Obtener la entrada y predicción del usuario
input = Variable(training_set[user_id]).unsqueeze(0)
output = sae.forward(input).squeeze(0)

# Máscara de películas no vistas
unseen_mask = (test_set[user_id] == 0)

# Obtener las puntuaciones y los índices de películas no vistas
unseen_predictions = output[unseen_mask].detach().numpy()
unseen_movies_idx = np.where(unseen_mask.numpy())[0]

# Ordenar las predicciones en orden descendente
sorted_indices = np.argsort(unseen_predictions)[::-1]

# Obtener el top 10 de recomendaciones
top_10_indices = unseen_movies_idx[sorted_indices[:10]]

# Mapear los índices al título de las películas
top_10_movies = [(movie_id_to_title[movie_idx_to_id[idx]], unseen_predictions[sorted_indices[i]]) 
                 for i, idx in enumerate(top_10_indices)]

# Imprimir el top 10
print(f"Top 10 recomendaciones para el usuario {user_id}:")
for rank, (title, score) in enumerate(top_10_movies, start=1):
    print(f"{rank}. {title} (Score: {score:.2f})")

Top 10 recomendaciones para el usuario 259:
1. Dangerous Minds (1995) (Score: 4.45)
2. American President, The (1995) (Score: 4.41)
3. Waiting to Exhale (1995) (Score: 4.38)
4. Mr. Holland's Opus (1995) (Score: 4.37)
5. House of the Spirits, The (1993) (Score: 4.31)
6. Carlito's Way (1993) (Score: 4.29)
7. Dracula: Dead and Loving It (1995) (Score: 4.29)
8. Welcome to the Dollhouse (1995) (Score: 4.29)
9. Tom and Huck (1995) (Score: 4.28)
10. Sense and Sensibility (1995) (Score: 4.26)
